In [1]:
# import pip

In [2]:
# pip.main(['install','/datasets/home/34/234/cs291gbl/fastText/. --user'])

In [1]:
import numpy as np
import pip
import torch
from torch import nn
import fastText
import argparse
from torch.autograd import Variable
import io
from scipy import linalg
# import torch
print(torch.__version__)
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

0.3.1.post3


In [2]:
def load_pretrained_embeddings(path):
    """
    Reload pretrained embeddings from a text file.
    """
    word2id = {}
    vectors = []
    id2word={}
    emb_path =path
    _emb_dim_file = 300
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        for i, line in enumerate(f):
            if i == 0:
                split = line.split()
               
            else:
                word, vect = line.rstrip().split(' ', 1)
                
                word = word.lower()
                vect = np.fromstring(vect, sep=' ')
                if np.linalg.norm(vect) == 0:  # avoid to have null embeddings
                    vect[0] = 0.01
               
                word2id[word] = len(word2id)
                id2word[len(word2id)]=word
                vectors.append(vect[None])
            if len(word2id) >= 200000:
                break

    
    print("Loaded %i pre-trained word embeddings." % len(vectors))


    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.concatenate(vectors, 0)
    embeddings = torch.from_numpy(embeddings).float()
    embeddings = embeddings
    
    return embeddings,word2id,id2word


def load_fasttext_model(path):
    return fastText.load_model(path)

In [3]:
src_embedding,src_word2id,src_id2word=load_pretrained_embeddings("data/wiki.en.vec")
tgt_embedding,tgt_word2id,tgt_id2word=load_pretrained_embeddings("data/wiki.es.vec")


Loaded 200000 pre-trained word embeddings.
Loaded 200000 pre-trained word embeddings.


In [4]:
src_embeddings = nn.Embedding(200000, 300, sparse=True)
src_embeddings.weight.data.copy_(src_embedding)

tgt_embeddings = nn.Embedding(200000, 300, sparse=True)
tgt_embeddings.weight.data.copy_(tgt_embedding)

src_embeddings.cuda()
tgt_embeddings.cuda()

Embedding(200000, 300, sparse=True)

In [5]:
class Discriminator(nn.Module):

    def __init__(self, dis_hid_dim,dis_dropout,dis_input_dropout):
        super(Discriminator, self).__init__()

        self.emb_dim = 300
        self.dis_layers = 2
        self.dis_hid_dim = dis_hid_dim
        self.dis_dropout = dis_dropout
        self.dis_input_dropout = dis_input_dropout

        layers = [nn.Dropout(self.dis_input_dropout)]
        for i in range(self.dis_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.dis_hid_dim
            output_dim = 1 if i == self.dis_layers else self.dis_hid_dim
            layers.append(nn.Linear(input_dim, output_dim))
            if i < self.dis_layers:
                layers.append(nn.LeakyReLU(0.2))
                layers.append(nn.Dropout(self.dis_dropout))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        assert x.dim() == 2 and x.size(1) == self.emb_dim
        return self.layers(x).view(-1)

In [6]:

def Orthogonal(beta):
    W=mapping.weight.data
    mapping.weight.data.copy_((1 + beta) * W - beta * W.mm(W.transpose(0, 1).mm(W)))
    


In [7]:
#set up linear mapping,
mapping = nn.Linear(300, 300, bias=False)
mapping.weight.data.copy_(torch.eye(300))
modelDisc= Discriminator(2048,0,0.1)
mapping.cuda()
modelDisc.cuda()


# class Map(nn.Module):
    
#     def __init__(self,emb_dim):
#         super(Map, self).__init__()

#         self.layer=nn.Linear(emb_dim,emb_dim,bias=False)
#     def forward(self,x):
#         return self.layer(x)
# modelMap=Map(300)
# modelMap.cuda()


Discriminator(
  (layers): Sequential(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=300, out_features=2048, bias=True)
    (2): LeakyReLU(0.2)
    (3): Dropout(p=0)
    (4): Linear(in_features=2048, out_features=2048, bias=True)
    (5): LeakyReLU(0.2)
    (6): Dropout(p=0)
    (7): Linear(in_features=2048, out_features=1, bias=True)
    (8): Sigmoid()
  )
)

In [8]:
 def get_dis_xy(bs,volatile,src_embeddings,tgt_embeddings):
       
        mf = 75000
        src_ids = torch.LongTensor(bs).random_(mf)
        tgt_ids = torch.LongTensor(bs).random_(mf)
      
        src_ids = src_ids.cuda()
        tgt_ids = tgt_ids.cuda()

        # get word embeddings
        src_emb = src_embeddings(Variable(src_ids, volatile=True))
        tgt_emb = tgt_embeddings(Variable(tgt_ids, volatile=True))
        src_emb = mapping(Variable(src_emb.data, volatile=volatile))
        tgt_emb = Variable(tgt_emb.data, volatile=volatile)

        # input / target
        x = torch.cat([src_emb, tgt_emb], 0)
        y = torch.FloatTensor(2 * bs).zero_()
        y[:bs] = 1 - 0.1
        y[bs:] = 0.1
        y = Variable(y.cuda())

        return x, y

In [9]:
# weights=list(modelMap.parameters())
# weights[0].data.copy_(torch.eye(300))
# weights=list(modelMap.parameters())
# print((weights[0]))

In [10]:
linoptimizer = torch.optim.SGD(mapping.parameters(), lr=0.1)
discoptimizer= torch.optim.SGD(modelDisc.parameters(), lr=0.1)

loss_fn=torch.nn.BCELoss()



In [14]:
iterations_per_epoch=1000000
batch_size=32
epoch=1
val_score=-1

for _ in range(epoch):
      
    for _n in range(0,iterations_per_epoch,batch_size):
       
        for __ in range(5):
            modelDisc.train()
            mapping.eval()
            x,y=get_dis_xy(batch_size,True,src_embeddings,tgt_embeddings)
            y_pred = modelDisc(Variable(x.data))
            loss= loss_fn(y_pred, y)
            discoptimizer.zero_grad()
            loss.backward()
            discoptimizer.step()
          
       
        for __ in range(1):
            modelDisc.eval()
            mapping.train()
            x,y=get_dis_xy(batch_size,False,src_embeddings,tgt_embeddings)
            y_pred = modelDisc(x)
            loss2= loss_fn(y_pred,1- y)
            linoptimizer.zero_grad()
            loss2.backward()
#             for param in mapping.parameters():
#                 print(param.grad.data.sum())

            linoptimizer.step() 
#             Orthogonal(0.001)
        
        if _n%500==0:
#             weights=list(modelMap.parameters())
#             print((weights[0]))
            print(torch.norm(mapping.weight.data,2))
            print(_n," Discriminator Loss :  ",loss.data)
            print("Generator Loss : ", loss2.data)

    
    

17.3201847076
(0, ' Discriminator Loss :  ', 
 0.6664
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 0.7319
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
17.1055374146
(4000, ' Discriminator Loss :  ', 
 0.5527
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 0.8824
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.9674053192
(8000, ' Discriminator Loss :  ', 
 0.5991
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 0.9211
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
17.0922412872
(12000, ' Discriminator Loss :  ', 
 0.6090
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 0.8478
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
17.1505584717
(16000, ' Discriminator Loss :  ', 
 0.5775
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 0.9189
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
17.2247104645
(20000, ' Discriminator Loss :  ', 
 0.5626
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Genera

16.3598880768
(184000, ' Discriminator Loss :  ', 
 0.4570
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.4059
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3367996216
(188000, ' Discriminator Loss :  ', 
 0.4701
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.6159
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3401412964
(192000, ' Discriminator Loss :  ', 
 0.5000
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.3049
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.2949142456
(196000, ' Discriminator Loss :  ', 
 0.4274
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.6678
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3496322632
(200000, ' Discriminator Loss :  ', 
 0.5201
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.5439
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3193454742
(204000, ' Discriminator Loss :  ', 
 0.4644
[torch.cuda.FloatTensor of size 1 (GPU 0)

16.3470516205
(368000, ' Discriminator Loss :  ', 
 0.4617
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.7213
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.350692749
(372000, ' Discriminator Loss :  ', 
 0.4355
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.4300
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3523139954
(376000, ' Discriminator Loss :  ', 
 0.4319
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.7422
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3868522644
(380000, ' Discriminator Loss :  ', 
 0.4155
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.7397
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3763046265
(384000, ' Discriminator Loss :  ', 
 0.4648
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.7306
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.4022331238
(388000, ' Discriminator Loss :  ', 
 0.4394
[torch.cuda.FloatTensor of size 1 (GPU 0)]

16.2833099365
(552000, ' Discriminator Loss :  ', 
 0.4424
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.8648
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3265724182
(556000, ' Discriminator Loss :  ', 
 0.4461
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.7865
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3638286591
(560000, ' Discriminator Loss :  ', 
 0.4465
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.8322
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.3615875244
(564000, ' Discriminator Loss :  ', 
 0.3940
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.7254
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.4104995728
(568000, ' Discriminator Loss :  ', 
 0.4000
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.7211
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.4323635101
(572000, ' Discriminator Loss :  ', 
 0.4126
[torch.cuda.FloatTensor of size 1 (GPU 0)

16.1253509521
(736000, ' Discriminator Loss :  ', 
 0.4198
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.8044
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.0982990265
(740000, ' Discriminator Loss :  ', 
 0.4054
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.8544
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.0967731476
(744000, ' Discriminator Loss :  ', 
 0.4016
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.9197
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.0492630005
(748000, ' Discriminator Loss :  ', 
 0.4060
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.8574
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.0118312836
(752000, ' Discriminator Loss :  ', 
 0.3891
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 2.0753
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
15.9681310654
(756000, ' Discriminator Loss :  ', 
 0.4067
[torch.cuda.FloatTensor of size 1 (GPU 0)

16.0131397247
(920000, ' Discriminator Loss :  ', 
 0.4157
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 2.0420
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.0060596466
(924000, ' Discriminator Loss :  ', 
 0.3804
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.9831
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
15.9926424026
(928000, ' Discriminator Loss :  ', 
 0.3746
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 2.0971
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.0131778717
(932000, ' Discriminator Loss :  ', 
 0.3912
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 2.1349
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
16.0076904297
(936000, ' Discriminator Loss :  ', 
 0.3625
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
('Generator Loss : ', 
 1.9874
[torch.cuda.FloatTensor of size 1 (GPU 0)]
)
15.9540843964
(940000, ' Discriminator Loss :  ', 
 0.3691
[torch.cuda.FloatTensor of size 1 (GPU 0)

In [ ]:
# def build_dictionary(src_emb,tgt_emb):
#     src_emb = mapping(Variable(src_emb, requires_grad=False).cuda())
#     src_emb=torch.FloatTensor(src_emb.cpu().data.numpy()).cuda()
#     tgt_emb = tgt_emb.cuda()
#     src_emb = src_emb / src_emb.norm(2, 1, keepdim=True).expand_as(src_emb)
#     tgt_emb = tgt_emb / tgt_emb.norm(2, 1, keepdim=True).expand_as(tgt_emb)


# #         s2t_candidates = get_candidates(src_emb, tgt_emb, _params)
# #         t2s_candidates = get_candidates(tgt_emb, src_emb, _params)
#     dictionary= create_dictionary(src_emb, tgt_emb)
#     return dictionary

# dictionary=build_dictionary(src_embedding,tgt_embedding)

    

In [ ]:
def procrustes(src_emb,tgt_emb):
    A = src_emb[dictionary[:, 0]]
    B = tgt_emb[dictionary[:, 1]]
    M = B.transpose(0, 1).mm(A).cpu().numpy()
    U, S, V_t = linalg.svd(M, full_matrices=True)
    mapping.weight.data.copy_(torch.from_numpy(U.dot(V_t)).type_as(W))


In [ ]:
# for i in range(5):
#     procrustes(src_embeddings,tgt_embeddings)
#     evaluation_metric(src_embeddings,tgt_embeddings)
#     val=evaluation_metric(src_embeddings,tgt_embeddings)
#     if val_score<val:
#         torch.save(mapping.state_dict(), './mapping.pth')
    

In [ ]:
# # spanish_embeddings
# tgt_emb2=tgt_embeddings.cpu().numpy()
# # print(tgt_emd.shape)


# def get_nn(word_emb, tgt_emb, tgt_id2word, K=5):
  
#     scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
#     k_best = scores.argsort()[-K:][::-1]
#     for i, idx in enumerate(k_best):
#         print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [ ]:
# def create_map(words,language):
#     id2word={}
#     for word in words:
#         id=language.get_word_id(word)
#         id2word[id]=word
#     return id2word
# spanish_map=create_map(Swords,spanish)
        

In [ ]:
# cat_embedding=mapping(Variable(src_embeddings[src_word2id['same']],requires_grad=False).cuda()).cpu().data
# word_emd=cat_embedding.numpy()
# get_nn(word_emd,tgt_emb,tgt_id2word)


In [ ]:
# cat_embedding=tgt_embeddings[tgt_word2id['misma']]
# word_emd=cat_embedding.numpy()
# get_nn(word_emd,fake_src_embeddings.cpu().numpy(),src_id2word)


In [11]:
# fake_src_embeddings=(mapping(Variable(src_embedding, requires_grad=False).cuda())).cuda()
# tgt_embeddings=(tgt_embeddings).cuda()
mf=75000
src_ids = torch.arange(0, mf, out=torch.LongTensor())
print(src_ids)
src_ids = src_ids.cuda()


# get word embeddings
src_emb = src_embeddings(Variable(src_ids, volatile=True))

tgt_emb= tgt_embeddings(Variable(src_ids, volatile=True))
# tgt_emb = tgt_embeddings(Variable(tgt_ids, volatile=True))


     0
     1
     2
  ⋮   
 74997
 74998
 74999
[torch.LongTensor of size 75000]



In [12]:
fake_src_embeddings=torch.FloatTensor(src_emb.cpu().data.numpy())
# # type(fake_src_embeddings)
# # src_embedding=src_embeddings.cpu().detach().numpy()
from sklearn.decomposition import PCA
pca = PCA(n_components=2, whiten=True)  # TSNE(n_components=2, n_iter=3000, verbose=2)
pca.fit(np.vstack([fake_src_embeddings,tgt_emb ]))
print('Variance explained: %.2f' % pca.explained_variance_ratio_.sum())
# src_emb=fake_src_embeddings/fake_src_embeddings.norm(2, 1, keepdim=True).expand_as(fake_src_embeddings)
# tgt_emb = tgt_embeddings / tgt_embeddings.norm(2, 1, keepdim=True).expand_as(tgt_embeddings)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


def plot_similar_word(src_words, src_word2id, src_emb, tgt_words, tgt_word2id, tgt_emb, pca):

    Y = []
    word_labels = []
    for sw in src_words:
        Y.append(src_emb[src_word2id[sw]])
        word_labels.append(sw)
    for tw in tgt_words:
        Y.append(tgt_emb[tgt_word2id[tw]])
        word_labels.append(tw)

    # find tsne coords for 2 dimensions
    Y = pca.transform(Y)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]

    # display scatter plot
    plt.figure(figsize=(10, 8), dpi=80)
    plt.scatter(x_coords, y_coords, marker='x')

    for k, (label, x, y) in enumerate(zip(word_labels, x_coords, y_coords)):
        color = 'blue' if k < len(src_words) else 'red'  # src words in blue / tgt words in red
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points', fontsize=19,
                     color=color, weight='bold')

    plt.xlim(x_coords.min() - 0.2, x_coords.max() + 0.2)
    plt.ylim(y_coords.min() - 0.2, y_coords.max() + 0.2)
    plt.title('Visualization of the multilingual word embedding space')

    plt.show()
    

In [ ]:
src_words = ['dog', 'the', 'me', 'tennis', 'cat', 'conference']
tgt_words = ['perro', 'el', 'yo', u'tenis',  'gato', 'conferencia']

plot_similar_word(src_words,src_word2id,fake_src_embeddings.cpu().numpy(),tgt_words,tgt_word2id,tgt_embedding.cpu().numpy(),pca)
# # english.get_word_id

In [ ]:
# dico=create_dictionary(src_emb,tgt_emb)
# print(type(src_embedding))

In [ ]:
# fake_src_embeddings2=(mapping(Variable(src_embeddings, requires_grad=False).cuda())).cuda()

In [ ]:
# cat_embedding=mapping(Variable(src_embeddings[src_word2id['the']],requires_grad=False).cuda()).cpu().data
# word_emd=cat_embedding.numpy()
# get_nn(word_emd,tgt_embeddings.cpu().numpy(),tgt_id2word)